In [5]:
import pandas as pd
import json
import os
from collections import Counter
import re
from dateutil.parser import parse
import yaml
import sys
import glob
import ast
from datetime import datetime
from dataproc import preprocess
from tqdm import tqdm
import numpy as np
#import seaborn as sns
import gzip


HCpath='/HDD16TB/weisi/qa_Health_and_Personal_Care.json.gz'


#HCmultipath='/HDD16TB/weisi/QA_Health_and_Personal_Care.json.gz'
HCmetapath='/HDD16TB/weisi/meta_Health_and_Personal_Care.json.gz'



def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


HCdf = getDF(HCpath)
HCmetadf=getDF(HCmetapath)
#HCmetadf.fillna('x', inplace=True) 

#metadata=parse(HCmetapath) #dictionary

HCdf['answerTime'] = pd.to_datetime(HCdf['unixTime'], unit='s').dt.strftime('%Y-%m-%d')  #UnixTime has null val
HCdf = HCdf.dropna(subset=['answerTime'])
HCdf['year'] =  pd.to_datetime(HCdf['answerTime']).dt.year
#HCdf.fillna('x', inplace=True) 
'''
VGpath='/HDD16TB/weisi/qa_Video_Games.json.gz'
VGmetapath='/HDD16TB/weisi/meta_Video_Games.json.gz'

VGdf = getDF(VGpath)
VGmetadf=getDF(VGmetapath)
VGmetadf.fillna('x', inplace=True) 

'''

KeyboardInterrupt: 

In [ ]:
HCdf['id'] = range(1, len(HCdf) + 1)


In [ ]:
HCdf.head()

,questionType,asin,answerTime,unixTime,question,answer,answerType,year,id
0,open-ended,7293000154,2013-12-20,1.387526e+09,which model Norelcos does it fit?,"here are 3 norelco shavers this fits: 715 RL, ...",NaN,2013,1
1,open-ended,7293000154,2014-05-27,1.401174e+09,What replacement blades do I use for my 5801XL...,The best I can do from the info from Norelco i...,NaN,2014,2
2,yes/no,7293000154,2014-06-07,1.402124e+09,are these made by philips norelco?,Yes and they work great.each set last me one y...,Y,2014,3
3,open-ended,7293000154,2014-04-15,1.397545e+09,I have a philips HQ 6695 does it fit?,yes,NaN,2014,4
4,open-ended,7293000154,2014-03-22,1.395472e+09,I have a pt 730 and I would like to get replac...,"I have purchased replacement blades, it's not ...",NaN,2014,5


In [ ]:
HCmetadf.head()


,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,0077614992,This is an example product description.,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],NaN,NaN
1,0615208479,By now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,NaN,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",NaN,NaN
2,0615269990,What's wrong with your patient?Do all the symp...,Occam's Razor,http://ecx.images-amazon.com/images/I/51fH-ABe...,"{'also_bought': ['1935660152', '0071743979', '...",{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,NaN
3,0615315860,NaN,101 BlenderBottle Recipes Quick and Easy,http://ecx.images-amazon.com/images/I/21zOQu2Q...,"{'also_bought': ['B006VT9RBM', 'B0010JLMO8', '...",{'Health & Personal Care': 254068},[[Health & Personal Care]],NaN,NaN
4,0615406394,This is an example product description.,"Aphrodite Reborn - Women's Stories of Hope, Co...",http://ecx.images-amazon.com/images/I/51rJLgsi...,"{'also_bought': ['0966035232', '1421407205']}",{'Health & Personal Care': 377936},[[Health & Personal Care]],NaN,NaN


In [ ]:
HCbi = HCdf[HCdf['questionType'].isin(['yes/no'])]
HCoe = HCdf[HCdf['questionType'].isin(['open-ended'])]

tqdm.pandas()
HCbi.question = HCbi.question.progress_apply(lambda x: preprocess.fast_clean(x))
HCbi.question = HCbi.question.apply(lambda x: ' '.join(x))
HCbi.answer  = HCbi.answer.progress_apply(lambda x: preprocess.fast_clean(x))
HCbi.answer  = HCbi.answer.apply(lambda x: ' '.join(x))


  0%|          | 0/32100 [00:00<?, ?it/s]

100%|██████████| 32100/32100 [00:00<00:00, 78097.07it/s]
/tmp/ipykernel_292945/1418708582.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HCbi.question = HCbi.question.progress_apply(lambda x: preprocess.fast_clean(x))
/tmp/ipykernel_292945/1418708582.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HCbi.question = HCbi.question.apply(lambda x: ' '.join(x))
100%|██████████| 32100/32100 [00:00<00:00, 41611.93it/s]
/tmp/ipykernel_292945/1418708582.py:7: SettingWithCopyWarning: 
A value is trying to be 

In [ ]:
tqdm.pandas()
HCoe.question = HCoe.question.progress_apply(lambda x: preprocess.fast_clean(x))
HCoe.question = HCoe.question.apply(lambda x: ' '.join(x))
HCoe.answer  = HCoe.answer.progress_apply(lambda x: preprocess.fast_clean(x))
HCoe.answer  = HCoe.answer.apply(lambda x: ' '.join(x))


100%|██████████| 45457/45457 [00:00<00:00, 76215.50it/s]
/tmp/ipykernel_292945/4281773030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HCoe.question = HCoe.question.progress_apply(lambda x: preprocess.fast_clean(x))
/tmp/ipykernel_292945/4281773030.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HCoe.question = HCoe.question.apply(lambda x: ' '.join(x))
100%|██████████| 45457/45457 [00:01<00:00, 36758.76it/s]
/tmp/ipykernel_292945/4281773030.py:4: SettingWithCopyWarning: 
A value is trying to be 

In [6]:
HCmetadf.fillna('x', inplace=True) #HCmetadf['description'].isnull().any() is True, preprocess will have error if not filled



/tmp/ipykernel_292945/145665780.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'x' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  HCmetadf.fillna('x', inplace=True) #HCmetadf['description'].isnull().any() is True, preprocess will have error if not filled


In [8]:
tqdm.pandas()
HCmetadf.description= HCmetadf.description.progress_apply(lambda x: preprocess.fast_clean(x))
HCmetadf.description = HCmetadf.description.apply(lambda x: ' '.join(x))

100%|██████████| 263032/263032 [00:15<00:00, 17308.75it/s]


In [9]:
HCmetadf.head()

,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,0077614992,this is an example product description,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],x,x
1,0615208479,by now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,x,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",x,x
2,0615269990,what wrong with your patient do all the sympto...,Occam's Razor,http://ecx.images-amazon.com/images/I/51fH-ABe...,"{'also_bought': ['1935660152', '0071743979', '...",{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,x
3,0615315860,,101 BlenderBottle Recipes Quick and Easy,http://ecx.images-amazon.com/images/I/21zOQu2Q...,"{'also_bought': ['B006VT9RBM', 'B0010JLMO8', '...",{'Health & Personal Care': 254068},[[Health & Personal Care]],x,x
4,0615406394,this is an example product description,"Aphrodite Reborn - Women's Stories of Hope, Co...",http://ecx.images-amazon.com/images/I/51rJLgsi...,"{'also_bought': ['0966035232', '1421407205']}",{'Health & Personal Care': 377936},[[Health & Personal Care]],x,x


In [6]:
print('binary question by:',HCbi.groupby('year').size())

print('open-ended question by:',HCoe.groupby('year').size())

binary question by: year
2006        1
2007        8
2008        9
2009        3
2010       12
2011       19
2012      221
2013     7000
2014    19232
2015     5595
dtype: int64
open-ended question by: year
2006       24
2007       77
2008       85
2009       93
2010      124
2011      152
2012      527
2013    11203
2014    26430
2015     6742
dtype: int64


In [10]:
# Merge the dataframes on the 'asin' column using a left merge
HCbi_merged = pd.merge(HCbi, HCmetadf[['asin', 'description', 'title']], on='asin', how='left')
HCoe_merged = pd.merge(HCoe, HCmetadf[['asin', 'description', 'title']], on='asin', how='left')

HCbi_merged.head()

,questionType,asin,answerTime,unixTime,question,answer,answerType,year,id,description,title
0,yes/no,7293000154,2014-06-07,1.402124e+09,are these made by philips norelco,yes and they work great each set last me one year,Y,2014,3,for best results replace shaver head every yea...,Philips Norelco HQ56/52 Reflex Plus and Micro ...
1,yes/no,7884890364,2013-10-31,1.383203e+09,is there sell by date on these packages,there is no mfg date or expiry date on these b...,N,2013,7,maxell lr44 batteries 10 pack,Maxell LR44 Batteries 10 Pack
2,yes/no,7884890364,2015-02-21,1.424506e+09,are they the same as ag 13,yes,Y,2015,8,maxell lr44 batteries 10 pack,Maxell LR44 Batteries 10 Pack
3,yes/no,7884890364,2015-01-24,1.422086e+09,are these the same as the lr44gd,do believe they are,?,2015,9,maxell lr44 batteries 10 pack,Maxell LR44 Batteries 10 Pack
4,yes/no,7884890364,2013-12-24,1.387872e+09,do they have the new hologram package,orange and black package checked all the volta...,?,2013,11,maxell lr44 batteries 10 pack,Maxell LR44 Batteries 10 Pack


In [12]:
HCmetadf['description'].isnull().any()

False

In [13]:
folder_path = '/home/weisi/Temporal/data/amazonQA/HC'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
# Export the merged dataframe to a JSON file
HCbi_merged.to_json( os.path.join(folder_path, 'QA_amazonHC_YN_allyear.json'), orient='records')
HCoe_merged.to_json( os.path.join(folder_path, 'QA_amazonHC_OE_allyear.json'), orient='records')

# Remove rows where description is empty from the merged dataframes
HCbi_merged = HCbi_merged[HCbi_merged['description'].notna() & (HCbi_merged['description'] != '')]
HCoe_merged = HCoe_merged[HCoe_merged['description'].notna() & (HCoe_merged['description'] != '')]

print('binary question by:',HCbi_merged.groupby('year').size())

print('open-ended question by:',HCoe_merged.groupby('year').size())

binary question by: year
2006        1
2007        7
2008        9
2009        3
2010        9
2011       11
2012      181
2013     6219
2014    17395
2015     5022
dtype: int64
open-ended question by: year
2006       22
2007       64
2008       71
2009       83
2010      108
2011       99
2012      433
2013    10139
2014    24022
2015     6051
dtype: int64


In [14]:
'''HCbi_2013=HCbi_merged[HCbi_merged['year'] == 2013]
HCbi_2013 = HCbi_2013[HCbi_2013['description'].notna() & (HCbi_2013['description'] != '')]
HCbi_2013.to_json(os.path.join(folder_path, 'QA_amazonHC_YN_2013.jsonl'), orient='records', lines=True)'''

for year in range(2012, 2016):
    HCbi_year = HCbi_merged[HCbi_merged['year'] == year]
    HCoe_year = HCoe_merged[HCoe_merged['year'] == year]
    
    #filename: 'QA_amazonHC_YN_2013.json'...
    filenameYN = f'QA_amazonHC_YN_{year}.json'
    filenameOE = f'QA_amazonHC_OE_{year}.json'
    # save to json file
    HCbi_year.to_json(os.path.join(folder_path, filenameYN), orient='records') 
    HCoe_year.to_json(os.path.join(folder_path, filenameOE), orient='records')


-----------------------------open other dataset files----------------------------------

In [3]:
import json
import pandas as pd
with open('/home/weisi/Temporal/data/combined_data.json', 'r', encoding='utf-8') as f:
    HCqa=pd.read_json(f,lines=True)
HCqa['year'] =  pd.to_datetime(HCqa['answerTime']).dt.year
yearly_counts = HCqa.groupby('year').size()
print(yearly_counts)
HCqa.head()

year
2006        1
2007        8
2008        9
2009        3
2010       12
2011       19
2012      221
2013     7000
2014    19232
2015     5595
dtype: int64


,questionType,asin,answerTime,unixTime,question,answer,answerType,description,title,salesRank,price,brand,year
0,yes/no,7293000154,2014-06-07,1402124400,"[are, these, made, by, philips, norelco]","[yes, and, they, work, great, each, set, last,...",Y,For best results replace shaver head every 2 y...,"[philips, norelco, hq56, 52, reflex, plus, and...",{'Health & Personal Care': 77771},24.86,Philips,2014
1,yes/no,7884890364,2013-10-31,1383202800,"[is, there, sell, by, date, on, these, packages]","[there, is, no, mfg, date, or, expiry, date, o...",N,Maxell LR44 Batteries 10 Pack,"[maxell, lr44, batteries, 10, pack]",x,0.92,Maxell,2013
2,yes/no,7884890364,2015-02-21,1424505600,"[are, they, the, same, as, ag, 13]",[yes],Y,Maxell LR44 Batteries 10 Pack,"[maxell, lr44, batteries, 10, pack]",x,0.92,Maxell,2015
3,yes/no,7884890364,2015-01-24,1422086400,"[are, these, the, same, as, the, lr44gd]","[do, believe, they, are]",?,Maxell LR44 Batteries 10 Pack,"[maxell, lr44, batteries, 10, pack]",x,0.92,Maxell,2015
4,yes/no,7884890364,2013-12-24,1387872000,"[do, they, have, the, new, hologram, package]","[orange, and, black, package, checked, all, th...",?,Maxell LR44 Batteries 10 Pack,"[maxell, lr44, batteries, 10, pack]",x,0.92,Maxell,2013


In [2]:
import json
with open('/home/weisi/Temporal/mimic-top50.json', 'r', encoding='utf-8') as file:
    first_line = file.readline().strip()  
    first_entry = json.loads(first_line)

print(first_entry)

{'uid': '10000032', 'did': '10000032-DS-21', 'text': 'name unit no admission date discharge date date of birth sex service medicine allergies no known allergies adverse drug reactions attending chief complaint worsening abd distension and pain major surgical or invasive procedure paracentesis history of present illness hcv cirrhosis ascites hiv on art ivdu copd bioplar ptsd presented from osh ed with worsening abd distension over past week pt reports self discontinuing lasix and spirnolactone weeks ago because she feels like they don do anything and that she doesn want to put more chemicals in her she does not follow na restricted diets in the past week she notes that she has been having worsening abd distension and discomfort she denies edema or sob or orthopnea she denies dysuria she had food poisoning week ago from eating stale cake 20 min after food ingestion which resolved the same day she denies other recent illness or sick contacts she notes that she has been noticing gum bleedi